In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/Skin_Disease_Dataset.zip" /content/


In [ ]:
!unzip -q "/content/Skin_Disease_Dataset.zip" -d /content/skin_dataset


In [ ]:
train_path = "/content/skin_dataset/skin Disease Dataset/kaggle/train"
val_path   = "/content/skin_dataset/skin Disease Dataset/kaggle/val"
test_path  = "/content/skin_dataset/skin Disease Dataset/kaggle/test"


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.amp import autocast, GradScaler

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cuda


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 16
NUM_CLASSES = 6

In [ ]:
!mkdir -p /content/drive/MyDrive/models


MobileNetv8

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = 6


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
Tesla T4


In [ ]:
import os
import shutil
import numpy as np
from torchvision import datasets

NUM_CLASSES = 6
SAMPLES_PER_CLASS = 1500

#train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train")
#targets = np.array(train_dataset.targets)

#balanced_dir = "/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced"
#os.makedirs(balanced_dir, exist_ok=True)

#for cls in range(NUM_CLASSES):
 #   cls_indices = np.where(targets == cls)[0]
 #   selected = np.random.choice(cls_indices, SAMPLES_PER_CLASS, replace=len(cls_indices)<SAMPLES_PER_CLASS)

 #   class_dir = os.path.join(balanced_dir, train_dataset.classes[cls])
 #   os.makedirs(class_dir, exist_ok=True)

 #   for idx in selected:
    #    src_path = train_dataset.imgs[idx][0]
    #    dst_path = os.path.join(class_dir, os.path.basename(src_path))
    #    shutil.copy(src_path, dst_path)


In [ ]:
import os
import shutil
import numpy as np
from torchvision import datasets

NUM_CLASSES = 6
SAMPLES_PER_CLASS = 3000

train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train")
targets = np.array(train_dataset.targets)

balanced_dir = "/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced"
os.makedirs(balanced_dir, exist_ok=True)

if os.path.exists(balanced_dir):
    shutil.rmtree(balanced_dir)

os.makedirs(balanced_dir)

for cls in range(NUM_CLASSES):
    cls_indices = np.where(targets == cls)[0]
    selected = np.random.choice(cls_indices, SAMPLES_PER_CLASS, replace=len(cls_indices)<SAMPLES_PER_CLASS)

    class_dir = os.path.join(balanced_dir, train_dataset.classes[cls])
    os.makedirs(class_dir, exist_ok=True)

    for j, idx in enumerate(selected):
     src_path = train_dataset.imgs[idx][0]

     filename = f"{j}_{os.path.basename(src_path)}"
     dst_path = os.path.join(class_dir, filename)

     shutil.copy(src_path, dst_path)


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_CLASSES = 6
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/val", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 18000
Validation set size: 3923


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model_mn = models.mobilenet_v2(weights="IMAGENET1K_V1")

for param in model_mn.features.parameters():
    param.requires_grad = False


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 180MB/s]


In [ ]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model_mn.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model_mn.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, NUM_CLASSES)
)


model_mn = model_mn.to(DEVICE)


In [ ]:
#from sklearn.utils.class_weight import compute_class_weight

#labels = np.array(train_dataset.targets)

#weights = compute_class_weight(
#    class_weight="balanced",
#    classes=np.unique(labels),
#    y=labels
#)

#class_weights = torch.tensor(weights, dtype=torch.float).to(DEVICE)
#print("Class weights:", class_weights)


In [ ]:
LR = 7e-6


#class_weights = class_weights.to(DEVICE)
#criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_mn.classifier.parameters(), lr=LR)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

EPOCHS = 30
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
best_val_f1 =0
best_val_recall = 0.0
counter = 0

MODEL_PATH = "/content/drive/MyDrive/models/skin_mobile_net_v2_1.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_mn.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_mn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_mn.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_mn(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_mn.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")
            break

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/30 | Batch 282/282 | Loss: 1.7443 | Acc: 0.3750 | Prec: 0.3194 | Rec: 0.2778 | F1: 0.2917


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 1 Summary
Train | Loss: 1.7767 | Acc: 0.2158 | Prec: 0.2328 | Rec: 0.2158 | F1: 0.2087
Val   | Acc: 0.4611 | Prec: 0.4696 | Rec: 0.4322 | F1: 0.3965
✔ Model improved. Saved.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...


Epoch 15 Summary
Train | Loss: 1.2167 | Acc: 0.5207 | Prec: 0.5201 | Rec: 0.5207 | F1: 0.5200
Val   | Acc: 0.5830 | Prec: 0.5356 | Rec: 0.6011 | F1: 0.5511
No improvement. Patience: 4/5


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 16/30 | Batch 282/282 | Loss: 0.9236 | Acc: 0.7500 | Prec: 0.7917 | Rec: 0.8333 | F1: 0.7484


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 16 Summary
Train | Loss: 1.2120 | Acc: 0.5201 | Prec: 0.5198 | Rec: 0.5201 | F1: 0.5196
Val   | Acc: 0.5858 | Prec: 0.5320 | Rec: 0.5986 | F1: 0.5495
No improvement. Patience: 5/5
🚨 Early stopping triggered


Fine Tuning by unfreezing last 2 layers of the base model.

In [ ]:
for param in model_mn.features[-2:].parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam([
    {"params": model_mn.classifier.parameters(), "lr": 5e-5},
    {"params": model_mn.features[-2:].parameters(), "lr": 1e-6}
])


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

EPOCHS = 15
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
best_val_f1 =0
best_val_recall = 0.0
counter = 0
MODEL_PATH = "/content/drive/MyDrive/models/skin_mobile_net_v2_2.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_mn.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_mn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_mn.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_mn(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_mn.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/15 | Batch 282/282 | Loss: 1.0323 | Acc: 0.5625 | Prec: 0.6111 | Rec: 0.5111 | F1: 0.5407


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 1 Summary
Train | Loss: 1.1915 | Acc: 0.5309 | Prec: 0.5315 | Rec: 0.5309 | F1: 0.5308
Val   | Acc: 0.6046 | Prec: 0.5500 | Rec: 0.6059 | F1: 0.5650
✔ Model improved. Saved.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...


Epoch 14 Summary
Train | Loss: 1.0051 | Acc: 0.6068 | Prec: 0.6112 | Rec: 0.6068 | F1: 0.6079
Val   | Acc: 0.6408 | Prec: 0.5833 | Rec: 0.6573 | F1: 0.6037
No improvement. Patience: 1/5


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 15/15 | Batch 282/282 | Loss: 1.1602 | Acc: 0.5625 | Prec: 0.5694 | Rec: 0.6726 | F1: 0.5546


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 15 Summary
Train | Loss: 0.9987 | Acc: 0.6099 | Prec: 0.6141 | Rec: 0.6099 | F1: 0.6108
Val   | Acc: 0.6498 | Prec: 0.5930 | Rec: 0.6558 | F1: 0.6126
✔ Model improved. Saved.


In [ ]:
model_mn = models.mobilenet_v2(weights="IMAGENET1K_V1")
model_mn.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model_mn.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, NUM_CLASSES)
)

model_mn.load_state_dict(torch.load("/content/drive/MyDrive/models/skin_mobile_net_v2_2.pth"))
model_mn = model_mn.to(DEVICE)


Fine Tuning by unfreezing last 4 layers of the base model

In [ ]:
for param in model_mn.features[-4:].parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {"params": model_mn.classifier.parameters(), "lr": 5e-5},
    {"params": model_mn.features[-3:].parameters(), "lr": 1e-6}
], weight_decay=1e-4)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

EPOCHS = 15
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
best_val_f1 =0
best_val_recall = 0.0
counter = 0
MODEL_PATH = "/content/drive/MyDrive/models/skin_mobile_net_v2_3.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_mn.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_mn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_mn.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_mn(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_mn.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/15 | Batch 282/282 | Loss: 0.8135 | Acc: 0.7500 | Prec: 0.6389 | Rec: 0.6000 | F1: 0.5694


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 1 Summary
Train | Loss: 0.9954 | Acc: 0.6107 | Prec: 0.6151 | Rec: 0.6107 | F1: 0.6118
Val   | Acc: 0.6561 | Prec: 0.5954 | Rec: 0.6704 | F1: 0.6119
Model improved. Saved.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...


Epoch 14 Summary
Train | Loss: 0.9239 | Acc: 0.6393 | Prec: 0.6430 | Rec: 0.6393 | F1: 0.6401
Val   | Acc: 0.6709 | Prec: 0.6103 | Rec: 0.6804 | F1: 0.6241
No improvement. Patience: 3/5


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 15/15 | Batch 282/282 | Loss: 1.0941 | Acc: 0.5000 | Prec: 0.5556 | Rec: 0.6488 | F1: 0.4901


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



Epoch 15 Summary
Train | Loss: 0.9162 | Acc: 0.6449 | Prec: 0.6493 | Rec: 0.6449 | F1: 0.6461
Val   | Acc: 0.6834 | Prec: 0.6203 | Rec: 0.6848 | F1: 0.6358
Model improved. Saved.
